In [29]:
from bs4 import BeautifulSoup
import requests, os
from concurrent.futures import ThreadPoolExecutor, as_completed
import pprint

In [2]:
booking_url = 'http://www.booking.com'
booking_country_url = 'http://www.booking.com/country'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [9]:
def get_countries():
    """
    :return: dictionary of all countries listed in booking.com as keys and their url as the value
    """
    try:
        global booking_country_url, booking_url
        page_results = requests.get(booking_country_url, headers=headers)
        if page_results:
            soup = BeautifulSoup(page_results.text, 'html.parser')
            country_lists = soup.find_all("li", {"class": "dest-sitemap__subsublist-item"}, "a")
            country_dict = {}
            for li in country_lists:
                for country in li.find_all("a"):
                    country_dict[country.text.strip()] = booking_url + country['href'].split('?')[0]
            return country_dict
        else:
            logger.error('request failed')
            return False
    except Exception as e:
        print(e)
        return False


def get_cities(country_url):
    """

    :param country_url: url of country
    :return: dictionary of all cities listed in a country as keys and their url as the value
    """
    try:
        global booking_url
        page_results = requests.get(country_url, headers=headers)
        if page_results:
            soup = BeautifulSoup(page_results.text, 'html.parser')
            cities_lists = soup.find_all("li", {"class": "dest-sitemap__subsublist-item"}, "a")
            cities_dict = {}
            for li in cities_lists:
                for city in li.find_all("a"):
                    cities_dict[city.text.strip()] = booking_url + city['href'].split('?')[0]
            return cities_dict
        else:
            logger.error('request failed')
            return False
    except Exception as e:
        print(e)
        return False

# edited to return number of hotels
def get_hotels(city_url):
    """

    :param city_url: url of city
    :return: dictionary of all hotels listed in a city as keys and their url as the value
    """
    
    try:
        global booking_url
        page_results = requests.get(city_url, headers=headers)
        if page_results:
            soup = BeautifulSoup(page_results.text, 'html.parser')
            city_results_lists = soup.find_all("li", {"class": "dest-sitemap__list-item"})
            hotels_dict = {}
            for city_result_li in city_results_lists:
                for header_3 in city_result_li.find_all("h3"):
                    for header in header_3:
                        if header.split(' ')[0] == 'Hotels' and header.split(' ')[1] == 'in':
                            #print(header)
                            hotels = city_result_li.find_all("li", {"class": "dest-sitemap__subsublist-item"}, "a")
                            print(header, len(hotels))
                            return len(hotels) # edit ------------
                            
#                             for li in hotels:
#                                 for hotel in li.find_all("a"):
#                                     hotels_dict[hotel.text.strip()] = booking_url + hotel['href'].split('?')[0]

            #return hotels_dict
#             print(count)
            return 0
        else:
            logger.error('request failed')
            return False
    except Exception as e:
        print(e)
        return False


In [128]:

def get_hotel_data(hotel_name, hotel_url):
    """

        :param hotel_name: hotel name
        :param hotel_url: url of hotel
        :return: dictionary of all cities listed in a country as keys and their url as the value
        """
    try:
        global booking_url
        page_results = requests.get(hotel_url, headers=headers)
        if page_results:
            soup = BeautifulSoup(page_results.text, 'html.parser')
            hotel_address = soup.find("span", {"class": "hp_address_subtitle js-hp_address_subtitle jq_tooltip"})
            hotel_description = []
            hotel_description_soup = soup.find("div", {"id": "property_description_content"})
            if hotel_description_soup:
                for p in hotel_description_soup.find_all("p"):
                    if not p.find("span"):
                        hotel_description.append(p.text)
            des = " ".join(hotel_description)
            all_facilities = {}
            facilities_lists = soup.find_all("div", {"class": "facilitiesChecklistSection"})
            #soup
            for facility_section in facilities_lists:
                facility_name = facility_section.find("h5").text.strip()
                all_facilities[facility_name] = []
                for li in facility_section.find_all("li"):
                    facility_with_span = li.find_all("span", {"data-name-en": True})
                    if facility_with_span:
                        for facility in facility_with_span:
                            all_facilities[facility_name].append(facility.text.strip().replace('\n', ''))
                    else:
                        all_facilities[facility_name].append(li.text.strip().replace('\n', ''))
                        
            all_most_popular_facilities = []
            most_popular_facilities = soup.find_all("div", {"class": "important_facility"}, "data-name-en")
            for facility in most_popular_facilities:
                all_most_popular_facilities.append(facility.text.strip().replace('\n', ''))
                
            all_property_surroundings = {}
            property_surroundings = soup.find_all("div", {"class": "hp_location_block__section_container"})
            for surrounding in property_surroundings:
                surrounding_name = surrounding.find("span")
                if surrounding_name:
                    for li in surrounding.find_all("li"):
                        location_name = li.find("div", {"class": "bui-list__description"}).text.strip().replace('\n', '')
                        location_distance = li.find("div", {"class": "bui-list__item-action hp_location_block__section_list_distance"}).text.strip().replace('\n', '')
                    
                        all_property_surroundings[surrounding_name.text.strip()] = {'location_name': location_name, 'location_distance': location_distance}
            score = float(soup.find("div", {"class": "e5a32fd86b"}).text)
            amount_of_reviews = float(soup.find("div", {"class": "_6a1b6ff88e"}).text.split(' ')[0])
            
            top_three_comments = []
            comments = soup.find("ul", {"class": "bui-carousel__inner"})
            if comments:
                lis = comments.find_all("div", {"class": "c-review-snippet"})
                for i in range(len(lis)):
                    if 2 < i:
                        break
                    name = lis[i].find("span", {"class": "bui-avatar-block__title"}).text.strip()
                    comment = lis[i].find("span", {"class": "c-review__body"}).text.strip()
                    top_three_comments.append({'name': name, 'comment': comment})
                    
            geometry = {}
            coordinates_fetch = soup.find("a", {"id": "hotel_sidebar_static_map"})["data-atlas-latlng"].split(',')
            if coordinates_fetch:
                geometry['type'] = 'Point'
                geometry['coordinates'] = [float(coordinates_fetch[1]), float(coordinates_fetch[0])]

                    
                
            return {'type': 'Feature',
                    'geometry': geometry,
                    'properties': {
                        'name': hotel_name,
                        'address': hotel_address.text.strip().replace('\n', ''),
                        'description': des,
                        'facilities': all_facilities,
                        'most_popular_facilities': list(set(all_most_popular_facilities)),
                        'property_surroundings': all_property_surroundings,
                        'score': score,
                        'amount_of_reviews': amount_of_reviews,
                        'top_three_comments': top_three_comments
                    }
                    
                   }
        else:
            logger.error('request failed')
            return False
    except Exception as e:
        print(e)
        return False

In [10]:
def get_all_in_country(country):
    all_countries = get_countries()
    if country not in all_countries:
        return False
    all_cities = get_cities(all_countries[country])
    processes = []
    with ThreadPoolExecutor() as executer:

        for city in all_cities.keys():
            processes.append(executer.submit(get_hotels, all_cities[city]))
    hotels = []
    for task in as_completed(processes):
        if task.result():
            hotels.append(task.result())

    return sum(hotels)

In [11]:
germa = get_all_in_country('Germany')

Hotels in Aalen 42
Hotels in Absberg 6
Hotels in Abenberg 4
Hotels in Abstatt 8
Hotels in Abtsroda 2
Hotels in Aachen 103
Hotels in Abensberg 4
Hotels in Accumersiel 1
Hotels in Achkarren 3
Hotels in Achslach 11
Hotels in Achern 6
Hotels in Achim 5
Hotels in Achstetten 1
Hotels in Acht 1
Hotels in Achberg 8
Hotels in Achterstadt 2
Hotels in AchthalHotels in Adamswalde 1
 1
Hotels in Adelberg 2
Hotels in Adelhausen 2
Hotels in Adelsheim 1
Hotels in Adelsdorf 5
Hotels in Adelheidsdorf 1
Hotels in Adelschlag 1
Hotels in Adelmannsfelden 1
Hotels in Adelshofen 2
Hotels in Adenau 44
Hotels in Adelsried 2
Hotels in Adorf 5
Hotels in Adendorf 4
Hotels in Admannshagen-Bargeshagen 1
Hotels in Ahaus 3
Hotels in Ahausen 2
Hotels in Agathenburg 1
Hotels in Affoldern 3
Hotels in Ahden 1
Hotels in AerzenHotels in Ahlbeck 5
 3
Hotels in Ahlefeld 3
Hotels in Ahlerstedt 2
Hotels in Ahlsdorf 1
Hotels in Ahlen 6
Hotels in Ahneby 1
Hotels in Ahnsen 1
Hotels in Ahnsbeck 1
Hotels in Ahorn 1
Hotels in Ahornta

Hotels in Bad Bergzabern 18
Hotels in Bad BelzigHotels in Bad Berleburg  24
7
Hotels in Bad Bertrich 36
Hotels in Bad Bevensen 43
Hotels in Bad Bibra 4
Hotels in Bad Bocklet 9Hotels in Bad Blankenburg 
8
Hotels in Bad Birnbach 77
Hotels in Bad Bodenteich 4
Hotels in Bad Brückenau 8
Hotels in Bad Boll 3
Hotels in Bad Brambach 10
Hotels in Bad Bramstedt Hotels in Bad Breisig 714

Hotels in Bad Buchau 16
Hotels in Bad Colberg 1
Hotels in Bad Camberg 18
Hotels in Bad DitzenbachHotels in Bad Doberan 8
 29
Hotels in Bad Dürkheim 24
Hotels in Bad DübenHotels in Bad Driburg  28
6
Hotels in Bad Dürrenberg 8
Hotels in Bad Eilsen 5
Hotels in Bad Dürrheim 34
Hotels in Bad Elster 49
Hotels in Bad Ems 25
Hotels in Bad Emstal 5
Hotels in Bad Endbach 11
Hotels in Bad Essen 6
Hotels in Bad Endorf 29
Hotels in Bad Feilnbach 28
Hotels in Bad Friedrichshall 10
Hotels in Bad Fallingbostel 19
Hotels in Bad Freienwalde 3
Hotels in Bad Frankenhausen 24
Hotels in Bad Gandersheim 4
Hotels in Bad Füssing 137
Hot

Hotels in Bergkamen 2
Hotels in Berghofen 1
Hotels in Bergholz 1
Hotels in Berghülen 2
Hotels in Bergisch Gladbach 51
Hotels in Berglern 3
Hotels in Berglase 1
Hotels in Bergneustadt 5
Hotels in Bergstein 1
Hotels in Bergrheinfeld 2
Hotels in Berkenbrück 1
Hotels in Berkenthin 2
Hotels in Bergweiler 1
Hotels in BerlinchenHotels in Berkheim 1
Hotels in Berlingen  42

Hotels in Berlingerode 1
Hotels in Bermersheim 1
Hotels in Berlin 1386
Hotels in Bermatingen 18
Hotels in Bermel 1
Hotels in Bermuthshain 1
Hotels in Bernau im Schwarzwald 26
Hotels in Bernau bei Berlin 9
Hotels in Bernau am Chiemsee 43
Hotels in Bernbeuren 5
Hotels in Berne 4Hotels in Bernburg
 22
Hotels in Berndorf 1
Hotels in Berneburg 3
Hotels in Berneck 1
Hotels in Bernhardsthal 1
Hotels in Bernkastel-Kues 144
Hotels in Bernitt 1
Hotels in Bernlohe 1
Hotels in Bernried 4
Hotels in Bersenbrück 2
Hotels in Berolzheim 2
Hotels in Bernsbach 3
Hotels in Bernshausen 2
Hotels in Bertingen 1
Hotels in Bersteland 1
Hotels in Be

Hotels in Breckerfeld 2
Hotels in Bredenborn 6
Hotels in Bredenfelde 1
Hotels in Breege 122
Hotels in Breese 1
Hotels in Breetz 1
Hotels in Brehme 2
Hotels in Brehna 2
Hotels in Breidenbach 1
Hotels in Breiholz 7
Hotels in Breisach am Rhein 12
Hotels in Breitenbach am Herzberg 1
Hotels in Breitbrunn am Chiemsee 15
Hotels in Breitenbach 1
Hotels in Breitenfelde 1
Hotels in Breitenberg 5
Hotels in Breitenbrunn 4
Hotels in Breitengüßbach 1
Hotels in Breitenthal 1
Hotels in Breitenstein 1
Hotels in Breitnau Hotels in Breitenworbis14 4

Hotels in Breitscheid 1
Hotels in Brekendorf 1
Hotels in Breitungen 3
Hotels in Breklum 4
Hotels in Bremen-Vegesack 4
Hotels in Bremerhaven 167
Hotels in Bremervörde 7
Hotels in Bremm 18
Hotels in Brennberg 2
Hotels in Bremke 3
Hotels in Brenkenhagen 4
Hotels in BrendenHotels in Brensbach  1
2
Hotels in Brenig 5
Hotels in Bresewitz 7
Hotels in Bresinchen 1
Hotels in Brennersgrün 1
Hotels in Brettach 1
Hotels in Bretten 5
Hotels in Bretzenheim 3
Hotels in Bri

Hotels in Dänschendorf Hotels in Dargelin12 2

Hotels in Darscheid 3
Hotels in Dargun 24
Hotels in Darsikow 1
Hotels in Darmstadt 54
Hotels in Darlingerode 5
Hotels in Dasbach 1
Hotels in Daskow 1
Hotels in Daschow 2
Hotels in Dassel 4
Hotels in Dassendorf 1
Hotels in Dasing 2
Hotels in Dassow 8
Hotels in Dattenfeld 3
Hotels in Dattenberg 1
Hotels in Datteln 3
Hotels in Datterode 1
Hotels in DaunHotels in Dauchingen 35
 1
Hotels in DausenauHotels in Debring  1
1
Hotels in Dechendorf 1
Hotels in Deckenpfronn 2
Hotels in Dedelstorf 2
Hotels in Deensen 2
Hotels in Degelstein 2
Hotels in Deesberg 1
Hotels in Deggenhausertal 16
Hotels in Degenershausen 1
Hotels in Deggendorf 16
Hotels in Degerfelden 1
Hotels in Deggingen 1
Hotels in Deibow 1
Hotels in DeidesheimHotels in Dehringhausen 1
 17
Hotels in Deilingen 1
Hotels in Deiningen 2
Hotels in Deinste 1
Hotels in Deisfeld 1
Hotels in Delitzsch 8
Hotels in Delbrück 6
Hotels in DeißlingenHotels in Deizisau  2
2
Hotels in Delliehausen 2
Hotels

...

Hotels in Sigmaringen 9
Hotels in Sigmarszell 12
Hotels in Silberborn 1
Hotels in Silberg 1
Hotels in Simbach am Inn 4
Hotels in Silberstedt 11
Hotels in Simbach 1
Hotels in Simmelsdorf 1
Hotels in Silz 2
Hotels in Simmern 7
Hotels in Simmersfeld 5
Hotels in Simmerath 43
Hotels in Simmershofen 2
Hotels in Simmertal 1
Hotels in SimonsbergHotels in Simonskall 18
 1
Hotels in Simonswald 27
Hotels in Simprechtshausen 1
Hotels in Sindelfingen 36
Hotels in Sindringen 2
Hotels in Sindelsdorf 2
Hotels in Sinntal Hotels in Singen2
 33
Hotels in Sinning 1
Hotels in Sinsheim 39
Hotels in Sinspert Hotels in Sinspelt1
Hotels in Sipplingen 37
 1
Hotels in Sinzheim 4
Hotels in Sinzig 9
Hotels in Sitzendorf 1
Hotels in Sinzing 6
Hotels in Siptenfelde 1
Hotels in Sittensen 4
Hotels in Sitzerath 1
Hotels in Sobrigau 1
Hotels in Sögel 30
Hotels in Sohland 3
Hotels in Soderstorf 3
Hotels in Söchtenau 1
Hotels in Soest 14
Hotels in Söhlde 1
Hotels in Sohra 1
Hotels in Sölden 1
Hotels in Solingen 48
Hotels 

Hotels in Sulzbach an der Murr 2
Hotels in Sulzbach 5
Hotels in Sulzdorf an der Lederhecke 3
Hotels in Sülzetal 1
Hotels in Sulzberg 6
Hotels in Sulzbach-Rosenberg 7
Hotels in Sulzburg 8
Hotels in Sulzfeld (im Grabfeld) 7
Hotels in Sulzfeld am Main 7
Hotels in Sülzhayn 1
Hotels in Sumte 1
Hotels in Sundhagen 4
Hotels in Sundhausen 1
Hotels in Sulzheim 2
Hotels in Sundern 20
Hotels in Süpplingenburg 2
Hotels in Sünna 1
Hotels in Süplingen 1
Hotels in Surwold Hotels in Süsel5 9

Hotels in Suppingen 1
Hotels in Süssau 31
Hotels in Suxdorf Hotels in Süßen2 5

Hotels in SykeHotels in Swisttal 3
 3
Hotels in Taching am See 9
Hotels in Taarstedt 2
Hotels in Tabarz 12
Hotels in Tailfingen 2
Hotels in Taben-Rodt 1
Hotels in Talheim 3
Hotels in Taltitz 3
Hotels in TammHotels in Tambach-Dietharz 12
 2
Hotels in TankenhagenHotels in Tangermünde 32
 1
Hotels in Tann 3
Hotels in Tanneberg Hotels in Tanna1
 11
Hotels in Tanne 19
Hotels in Tannenberg 3
Hotels in Tännesberg 10
Hotels in Tanzfleck 1
Hot

Hotels in Untertalheim 1
Hotels in Unterspreewald 1
Hotels in Untersteinbach 1
Hotels in Unterthingau 2
Hotels in Unteruhldingen 2
Hotels in Unterweißbach 3
Hotels in Unterwellenborn 4
Hotels in Unterumbach 1
Hotels in Unterweid 2
Hotels in Unterwittighausen 1
Hotels in Unterwössen 39
Hotels in Unterwolfert 1
Hotels in Untrasried 6
Hotels in Unzenberg 2
Hotels in Uplengen 18
Hotels in UplewardHotels in Upgant-Schott 10
 1
Hotels in Uphusum 3
Hotels in Urach 2
Hotels in Upost 1
Hotels in Urbach 3
Hotels in Urbar 4
Hotels in Urberg 8
Hotels in Urmitz 2
Hotels in Urbar-Mayen-Koblenz 1
Hotels in Ursberg 1
Hotels in Urnshausen 5
Hotels in Ursensollen 4
Hotels in Ürzig 18
Hotels in Usedom Town 10
Hotels in Uschlag 1
Hotels in Userin 19
Hotels in Utarp 15
Hotels in Usingen 13
Hotels in Uslar 15
Hotels in Ustersbach 1
Hotels in Utersum 31
Hotels in Uttenweiler 1
Hotels in Utlandshörn 1
Hotels in Utting am Ammersee 2
Hotels in Utzenfeld 1
Hotels in Uttewalde 1
Hotels in Utting 4
Hotels in Utzer

Hotels in Weida 3
Hotels in Weibersbrunn 4
Hotels in Wehrheim 2
Hotels in Weibern 7
Hotels in Weidelbach 1
Hotels in Weiden 14
Hotels in Weidenbach 2
Hotels in Weidenhain 1
Hotels in Weidenberg 6
Hotels in Weiersbach 2
Hotels in Weiding 1
Hotels in Weierweiler 1
Hotels in Weidhausen bei Coburg 1
Hotels in Weigelsberg 1
Hotels in Weigendorf 1
Hotels in Weigenheim 2
Hotels in Weihenstephan 1
Hotels in Weihenzell 1
Hotels in Weiherhammer 3
Hotels in Weil im Schönbuch 3
Hotels in Weil am Rhein 41
Hotels in Weikersheim 7
Hotels in Weil der Stadt 2
Hotels in Weilbach 3
Hotels in Weilburg 6
Hotels in Weiler-Simmerberg 10
Hotels in Weiler 1
Hotels in Weilerbach 3
Hotels in Weilerswist 4
Hotels in Weilheim 19Hotels in Weilmünster 
3
Hotels in Weilheim an der Teck 6
Hotels in Weilheim in Oberbayern 8
Hotels in Weilrod 1
Hotels in Weiltingen 1
Hotels in Weinbach 1
Hotels in Weimar 117
Hotels in Weinähr 1
Hotels in Weinböhla 2
Hotels in Weingarten 10
Hotels in Weinheim 19
Hotels in Weinstadt 13
Ho

Hotels in WittstockHotels in Wittnau 1
 4
Hotels in Wittmund 29
Hotels in Witzelroda 1
Hotels in WitzhaveHotels in Witzenhausen 2
 20
Hotels in Witzmannsberg 3
Hotels in Witzwort 10
Hotels in Wobbenbüll 3
Hotels in Wohlau 2
Hotels in Wohlenhagen 1
Hotels in Wohlbach 1
Hotels in Wohlenberg 28
Hotels in Wokuhl 1
Hotels in Wohlsborn 1
Hotels in Wöhrden 2
Hotels in Woldegk 2
Hotels in Wolf 2
Hotels in Woldzegarten 4
Hotels in Wolfersdorf 3
Hotels in Wolfach 20
Hotels in Wolfegg 7
Hotels in Wolfen 1
Hotels in Wolfenbüttel 22
Hotels in Wolframs-Eschenbach 10
Hotels in Wölfersheim 2
Hotels in Wolfhagen 10
Hotels in Wolfratshausen 4
Hotels in Wolfsberg 1
Hotels in Wolfsburg 50
Hotels in Wolfsburg-Unkeroda 2
Hotels in Wolfshagen 15
Hotels in Wolfschlugen 4
Hotels in Wolfsegg 1
Hotels in Wolfsgrund 1
Hotels in Wolfshausen 1
Hotels in Wolfstein 2
Hotels in Wolfsheim 1
Hotels in Wolfskrug 1
Hotels in Wolgast 19
Hotels in Wolgasterfähre 2
Hotels in Wolpadingen 2
Hotels in Wolkenstein 12
Hotels in W

In [13]:
print(germa)

96737


In [51]:
isra = get_all_in_country('Israel')

Hotels in ‘Eilabūn
Hotels in ‘AseretHotels in ‘Amqa

Hotels in ‘Ein Ya‘aqov
Hotels in ‘Ein el AsadHotels in ‘Agur

Hotels in ‘Afula
Hotels in ‘Avdon
Hotels in ‘Akko
Hotels in ‘En DorHotels in Abirim

Hotels in ‘En H̱arod
Hotels in ‘En Kerem
Hotels in ‘Ezuz
Hotels in ‘Isfiyā
Hotels in ‘En Hod
Hotels in ‘En Yahav
Hotels in AdamitHotels in Afik

Hotels in Adi
Hotels in Alsira
Hotels in Amikam
Hotels in Amirim
Hotels in Abū Ghaush
Hotels in Almog
Hotels in Aẖihud
Hotels in Alonei Habashan
Hotels in Ammiad
Hotels in Arad
Hotels in Ashdod
Hotels in Ani'am
Hotels in Ashalim
Hotels in Ashkelon
Hotels in Ashdot Ya‘aqov
Hotels in Ari'el
Hotels in Assaf Harofeh
Hotels in Arbel
Hotels in Atlit
Hotels in Avnei Eitan
Hotels in Bat Shelomo
Hotels in Bat Yam
Hotels in Be'er Milka
Hotels in Beʼer Ora
Hotels in Beer Sheva
Hotels in Beit Jann
Hotels in Bet Alfa
Hotels in Ben ‘Ammi
Hotels in Bene Beraq
Hotels in Bet HaKerem
Hotels in Bet H̱anan
Hotels in Bet HaShitta
Hotels in Bet H̱ananya
Hotels in Bet H

In [6]:
print(isra)

NameError: name 'isra' is not defined

In [129]:
arb_data = get_hotel_data('Arbel Guest House Shavit Family', 'http://www.booking.com/hotel/il/arbel-guest-house.en-gb.html')

In [126]:
pprint.pprint(arb_data)
print(arb_data)

{'geometry': {'coordinates': [32.81346258, 35.48308502], 'type': 'Point'},
 'properties': {'address': 'Arbel Village Lower Galillee, Arbel, 15282, Israel',
                'amount_of_reviews': 652.0,
                'description': 'The in-house chef prepares the extensive '
                               'breakfast and evening meals in front of '
                               'diners. Meals are served in the bright dining '
                               'room which overlooks the garden, and there are '
                               "also barbecue facilities. Located 10 minutes' "
                               'drive from the historic town of Tiberias and '
                               'the Sea of Galilee, Guest House Arbel offers '
                               'free parking. ',
                'facilities': {'Activities': ['Hiking',
                                              'Library',
                                              'Hot tub/Jacuzzi',
                         

In [130]:
import json 
print(json.dumps(arb_data))

{"type": "Feature", "geometry": {"type": "Point", "coordinates": [35.48308502, 32.81346258]}, "properties": {"name": "Arbel Guest House Shavit Family", "address": "Arbel Village Lower Galillee, Arbel, 15282, Israel", "description": "Surrounded by gardens and fruit trees, Arbel Guest House features an outdoor swimming pool and a sun terrace. The spacious accommodation has a hydromassage bath and free Wi-Fi. The in-house chef prepares the extensive breakfast and evening meals in front of diners. Meals are served in the bright dining room which overlooks the garden, and there are also barbecue facilities. Located 10 minutes' drive from the historic town of Tiberias and the Sea of Galilee, Guest House Arbel offers free parking. ", "facilities": {"Outdoors": ["Outdoor furniture", "Sun terrace", "BBQ facilities", "Terrace", "Garden"], "Pets": ["Pets are not allowed."], "Activities": ["Hiking", "Library", "Hot tub/Jacuzzi", "Massage"], "Food & Drink": ["Chocolate or cookies", "Bottle of water

In [ ]:
def add_hotel_data()